In [531]:
import face_recognition
import cv2
import os
import numpy as np

class Image_Preprocessing:
    def __init__(self):
        pass
    def face_preprocessing(self, image_path, path_bool = True):
        if path_bool:
            return self.crop_rotate(image_path, path_bool)
        else :
            return self.crop_rotate(image_path, path_bool)

    # 얼굴 영역별로 각 사진을 회전시키고
    # 회전된 얼굴만 잘라냄(랜드마크와 얼굴검출영역 최대값으로 얼굴이 최대한 온전히 나오게)
    def crop_rotate(self, img_path, path_bool):
        if path_bool:
            image = cv2.imread(img_path)
        else :
            image = img_path
        face_landmarks = face_recognition.face_landmarks(image)

        image_list = []
        # 각 얼굴들 검출해서 회전시킬 중심점들을 찾아서 (몇번째 얼굴, 회전 중심)의 dict형태로 추가
        for landmarks in face_landmarks:
            left_eye_center = np.array(landmarks['left_eye']).mean(axis=0)
            right_eye_center = np.array(landmarks['right_eye']).mean(axis=0)
            lec_x = int(left_eye_center[0])
            lec_y = int(left_eye_center[1])
            rec_x = int(right_eye_center[0])
            rec_y = int(right_eye_center[1])
            # 눈의 중앙 좌표와 각을 구함
            angle = np.degrees(np.arctan2((lec_y - rec_y), (lec_x - rec_x))) - 180
            # print("left_eye_center : {} right_eye_center : {}".format((lec_x, lec_y), (rec_x, rec_y)))
            # print("angle : {}".format(angle))
            # 회전 중심 점
            center_rotate = [int((lec_x + rec_x) / 2), int((lec_y + rec_y) / 2)]
            M = cv2.getRotationMatrix2D((center_rotate[0], center_rotate[1]), angle, 1)
            image_list.append(self.crop_image(image, center_rotate, M))

        return image_list

    # 두 점 사이의 거리
    def distacne_two_point(self, list_point1, list_point2):
        dx = list_point1[0] - list_point2[0]
        dy = list_point1[1] - list_point2[1]
        # 굳이 소수점까지 볼 필요는 없음
        return int((dx ** 2 + dy ** 2) ** 0.5)

    # 각 이미지로 부터 회전 중심값과 회전 설정 값을 받아
    # 이미지를 회전시키고 그 중 보내진 이미지의 얼굴영역과 가장 가까운 얼굴을 도려냄
    def crop_image(self, image, center_rotate, M):
        rotate_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        face_landmarks = face_recognition.face_landmarks(rotate_image)
        distance_list = []

        for landmark in face_landmarks:
            left_eye_center = np.array(landmark['left_eye']).mean(axis=0)
            right_eye_center = np.array(landmark['right_eye']).mean(axis=0)
            eye_center = ((left_eye_center + right_eye_center) / 2).astype('uint16')
            # 회전된 이미지의 축은 이전 함수에서 해당하는 얼굴의 중심점이란걸 알 수 있다
            # 이 점과 그리고 현재 회전된 이미지에서의 landmark의 중심점 사이의 거리가 가장 짧은 얼굴이
            # 해당하는 얼굴이니 이 얼굴을 도려내야함
            # print("center_rotate : {} , eye_center : {}".format(center_rotate, eye_center))
            distance_list.append(self.distacne_two_point(eye_center, center_rotate))

        # distance가 가장 짧은 index를 가지고 옴
        for i, d in enumerate(distance_list):
            if d == min(distance_list):
                crop_image_index = i
        # print("distance_list : {}".format(distance_list))
        # print("crop_image_index : {}".format(crop_image_index))
        # print("len(face_landmarks) : {}".format(len(face_landmarks)))
        current_face_mark = face_landmarks[crop_image_index]

        landmarks_list = []
        for k in current_face_mark.keys():
            landmarks_list.extend(current_face_mark[k])
        landmarks_array = np.array(landmarks_list)
        rect_list = []
        # 얼굴 rect를 x,y좌표로 구분해놓고
        for array in np.split(np.array(face_recognition.face_locations(rotate_image)[crop_image_index]), 2):
            rect_list.append(array)
        # landmkar와 rect에 대한 x,y,좌표를 np.array로 만들어서 최소값 최대값을 쉽게 뽑도록 함
        for i in range(len(rect_list)):
            temp = rect_list[i][0]
            rect_list[i][0] = rect_list[i][1]
            rect_list[i][1] = temp
            landmarks_array = np.vstack((landmarks_array, rect_list[i].reshape((-1, 2))))
        min_xy = landmarks_array.min(axis=0)
        max_xy = landmarks_array.max(axis=0)
        image = image[min_xy[1]:max_xy[1], min_xy[0]:max_xy[0]] 
        return cv2.resize(image, (96,96))


class Face_Embedding:
    # 생성자에서 db 만듬
    def __init__(self):
        self.create_db()
        self.img_preprocessing = Image_Preprocessing()

    def create_db(self):
        self.face_database = {}

    # image가 얼굴을 갖고 있는지 check
    def check_img(self, image_path, path_bool):
        if path_bool:
            img_ = cv2.imread(image_path)
        else:
            img_ = image_path

        if len(face_recognition.face_locations(img_)) == 0:
            return False
        return True

    # opencv의 경우는 rgb가아닌 bgr의 color이기 때문 plt로 출력할거 아니면 안바꿔도 됨
    def convert_rgb(self, image):
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # iamge path로 image를 각 얼굴에대해 회전하고 랜드마크를 포함하게 잘라내서 embedding 값 받음
    def get_face_embedding(self, image_path, path_bool = True):
        # 얼굴 없으면 0,0
        # print(image_path)
        if not self.check_img(image_path, path_bool):
            print('얼굴을 찾을 수 없습니다.')
            return 0
        face_encoding_list = []
        images = self.img_preprocessing.face_preprocessing(image_path, path_bool)
        for img in images:
            img_size = [(0, img.shape[1], img.shape[0], 0)]
            face_encoding = face_recognition.face_encodings(img, img_size)
            # list 형태로 return 하기 때문에
            face_encoding_list.append(face_encoding[0])

        return face_encoding_list

    # setup_db
    def setup_database(self, img_path):
        # setup 성공한 이미지 / 전체 이미지
        totla_image = len(os.listdir(img_path))
        count = 0
        not_face = []

        # ./test_images/test1/ 내의 모든 파일
        for p in os.listdir(img_path):
            # 파일 이름을 key로
            identity = str.split(p, '.')[0]
            print("file {} embedding...".format(p))
            # 해당 이미지 파일 경로를 _img로 해놓고
            img_ = os.path.join(img_path, p)
            # 해당 이미지에대한 embedding값 구해서 encodings에 넣어줌
            encodings = self.get_face_embedding(img_)
            # 실패한 파일들 볼 수 있도록 append
            if encodings == 0:
                not_face.append(p)
                continue
            elif len(encodings) == 1:
                self.face_database[identity] = encodings
            # 사진에 여러 얼굴 검출되면
            # [array1, array2] 처럼 하나의key에 각 얼굴들의 embedding 값을 넣어 줌
            else:
                self.face_database[identity] = [e for e in encodings]
            # encodings(embedding 값)은 128차원의 특징 값
            count += 1
            print("{} / {}".format(count, totla_image))
        print("얼굴을 못 읽는 이미지 : {}".format(not_face))

    # 이미지가 같은지 embedding된 값으로 비교
    # tolerance 이 더 낮으면 엄격함, 0.6을 추천함
    def compare_img(self, img_path, encoding_check_img, path_bool=True, tolerance=0.6):
        if path_bool:
            encoding_origin_img = self.get_face_embedding(img_path, path_bool)
        else:
            encoding_origin_img = img_path

        distance_diff = []

        if len(encoding_check_img) > 1:
            face_diff = []
            dist_diff = []
            # 여기서 encdoing_check_img는 이중list의 형태
            # [[np.array], [np.array]]
            for e in encoding_check_img:
                # 해당 embedding 값들과 비교하고자하는 original image 얼굴의 embedding값 비교
                # face_recognition.face_distance는 내부적으로
                # 두 벡터간의 차를 구하고 그 차에대해 벡터의 길이를 구함
                # 즉 길이가 짧을 수록(값이 작을 수록) 두 얼굴은 일치
                diff = face_recognition.face_distance(encoding_origin_img, e)
                # 벡터간 거리
                dist_diff.append(diff)
                print(diff)
                face_diff.append(list(diff <= tolerance))
            distance_diff.extend(min(dist_diff))
            print(face_diff)
            return distance_diff, [any(list(face_diff))]
        # 한 사진에 한 얼굴만 검출되면 np.array 타입으로 반환하기때문에 바로 비교
        # 첫번째 인자의 embedding값은 list형태로, 2번째 인자는 numpy 형태로 들어가줘야 함
        face_diff = face_recognition.face_distance(encoding_origin_img, encoding_check_img[0])
        print(face_diff)
        distance_diff.append(face_diff)
        return distance_diff, list(face_diff <= tolerance)
        # compare_faces는 내부적으로
        # 128차원의 embedding된 값의 차로 얼굴간의 거리를 구한뒤
        # np.linalg.norm를 통해 128차원의 벡터의 길이를 구함
        # 이길이가 tolerance보다 작을때 두 이미지는 같은 이미지로 간주
        # tolearnace의 값을 높이면 비교연산이 좀더 관대해지게됨

    # 들어온 이미지와 DB를 비교
    def compare_db(self, img_path, path_bool = True):
        bool_list = []
        for k in self.face_database.keys():
            _, compare_check = self.compare_img(img_path, self.face_database[k],path_bool, 0.6)
            if path_bool:
                if compare_check:
                    print('{}는 {}와 일치합니다'.format(os.path.split(img_path)[-1], k))
                else:
                    print('{}는 {}와 불 일치합니다'.format(os.path.split(img_path)[-1], k))
            else:
                if compare_check:
                    print('올린 이미지는 {}와 일치합니다'.format(k))
                else:
                    print('올린 이미지는는 {}와 불 일치합니다'.format(k))

            bool_list.extend(compare_check)
        return bool_list

    # 가장 유사한 이미지 return
    def similar_compare_db(self, img_path, path_bool = True):
        diff_list = []
        db_diff = {}
        for i, k in enumerate(self.face_database.keys()):
            db_diff[i] = k
            distance_diff, compare_check = self.compare_img(img_path, self.face_database[k],path_bool, 0.6)
            # print(distance_diff)
            # if compare_check:
            #     print('{}는 {}와 일치합니다'.format(os.path.split(img_path)[-1], k))
            # else:
            #     print('{}는 {}와 불 일치합니다'.format(os.path.split(img_path)[-1], k))
            diff_list.extend(distance_diff)
        min_diff = np.array(diff_list).argmin()
        print('{}가 {}만큼 가까워서 제일 유사한 얼굴'.format(db_diff[min_diff], diff_list[0]))
        return db_diff[min_diff]
    # 특정 경로 폴더의 이미지들 embedding 값들 csv로 저장
#     def embedding_to_csv(image_path):
        



In [533]:
test1 = Face_Embedding()
test1.setup_database('C:/TakeMeHome/IM/web_app/flaskapp/static/images')

image1 = cv2.imread('C:/TakeMeHome/IM/opencv_project/test_images/sin1.jpg')
emb1 = test1.get_face_embedding(image1, False)
test1.compare_db(emb1, False)

file kim10.jpg embedding...
1 / 11
file kim11.jpg embedding...
얼굴을 찾을 수 없습니다.
file kim12.jpg embedding...
얼굴을 찾을 수 없습니다.
file kim2.jpg embedding...
2 / 11
file kim3.jpg embedding...
3 / 11
file kim4.jpg embedding...
4 / 11
file kim5.jpg embedding...
5 / 11
file kim6.jpg embedding...
6 / 11
file kim7.jpg embedding...
7 / 11
file kim8.jpg embedding...
8 / 11
file kim9.jpg embedding...
9 / 11
얼굴을 못 읽는 이미지 : ['kim11.jpg', 'kim12.jpg']
[0.4571136]
올린 이미지는 kim10와 일치합니다
[0.47422084]
올린 이미지는 kim2와 일치합니다
[0.44124373]
올린 이미지는 kim3와 일치합니다
[0.43474695]
올린 이미지는 kim4와 일치합니다
[0.43098571]
올린 이미지는 kim5와 일치합니다
[0.43055617]
올린 이미지는 kim6와 일치합니다
[0.45303865]
올린 이미지는 kim7와 일치합니다
[0.47419133]
올린 이미지는 kim8와 일치합니다
[0.49147994]
올린 이미지는 kim9와 일치합니다


[True, True, True, True, True, True, True, True, True]

In [432]:
embedding_list = []
for k in test1.face_database:
    embedding_list.append(test1.face_database[k][0])
embedding_df = pd.DataFrame(embedding_list, index = list(test1.face_database.keys()))
embedding_df.to_csv('test.csv', index=True, header=True)

In [433]:
embedding_df

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
kim10,-0.084063,0.039731,0.048850,-0.139994,-0.132822,-0.030529,-0.079642,-0.133435,0.107183,-0.169251,...,0.131356,0.068678,-0.021161,0.044097,-0.167169,-0.023806,0.135986,-0.001535,0.121353,0.006514
kim2,-0.099417,0.086221,0.078085,-0.064725,-0.112766,-0.038902,-0.035343,-0.149757,0.097881,-0.174078,...,0.122097,0.083155,-0.029671,0.049072,-0.197699,-0.003710,0.147324,0.024033,0.092897,0.002944
kim3,-0.101092,0.052281,0.072207,-0.138972,-0.104183,-0.027339,-0.082474,-0.149408,0.124288,-0.177972,...,0.133492,0.100071,-0.002764,0.015872,-0.208168,-0.013111,0.141747,-0.008849,0.080920,-0.034918
kim4,-0.100174,0.018409,0.058266,-0.080150,-0.108256,-0.051156,-0.087436,-0.120186,0.028922,-0.170093,...,0.142973,0.100357,0.003116,0.006549,-0.198405,-0.037609,0.179810,-0.000526,0.096610,-0.008519
kim5,-0.091822,0.031022,0.069909,-0.088182,-0.150732,-0.048896,-0.048058,-0.147652,0.059332,-0.126718,...,0.109271,0.074506,-0.042370,0.061233,-0.185493,-0.043759,0.135925,-0.002170,0.070462,-0.012326
kim6,-0.100594,0.032856,0.047395,-0.085442,-0.062414,-0.080291,-0.103689,-0.138430,0.112533,-0.117098,...,0.158315,0.114222,-0.018195,0.036954,-0.196113,-0.025679,0.116657,-0.009947,0.122645,-0.023930
kim7,-0.141279,0.048912,0.047157,-0.140672,-0.160152,-0.018800,-0.106201,-0.078768,0.114847,-0.188819,...,0.105087,0.090598,-0.045077,0.003897,-0.263763,-0.033162,0.154792,0.024102,0.088644,0.021292
kim8,-0.089850,0.096005,0.041754,-0.052878,-0.094131,0.027970,-0.116759,-0.078475,0.174731,-0.163058,...,0.136254,0.108000,-0.029237,-0.003817,-0.210180,0.007962,0.089843,-0.071499,0.115876,-0.006083
kim9,-0.074607,0.043346,0.093928,-0.107175,-0.102226,-0.059615,-0.074609,-0.148097,0.136101,-0.160114,...,0.116255,0.082958,-0.035539,0.003622,-0.160167,0.003947,0.112970,0.025309,0.095867,-0.010663


In [434]:
embedding_df.to_csv('test.csv', index=True, header=True)

# -------------------- load

In [435]:
load_embedding_csv = pd.read_csv('test.csv')

In [436]:
file_name = list(load_embedding_csv.iloc[:,0])

In [437]:
load_embedding_csv.index = file_name

In [438]:
load_embedding_csv = load_embedding_csv.drop(load_embedding_csv.columns[0],1)

In [447]:
load_embedding_csv

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
kim10,-0.084063,0.039731,0.048850,-0.139994,-0.132822,-0.030529,-0.079642,-0.133435,0.107183,-0.169251,...,0.131356,0.068678,-0.021161,0.044097,-0.167169,-0.023806,0.135986,-0.001535,0.121353,0.006514
kim2,-0.099417,0.086221,0.078085,-0.064725,-0.112766,-0.038902,-0.035343,-0.149757,0.097881,-0.174078,...,0.122097,0.083155,-0.029671,0.049072,-0.197699,-0.003710,0.147324,0.024033,0.092897,0.002944
kim3,-0.101092,0.052281,0.072207,-0.138972,-0.104183,-0.027339,-0.082474,-0.149408,0.124288,-0.177972,...,0.133492,0.100071,-0.002764,0.015872,-0.208168,-0.013111,0.141747,-0.008849,0.080920,-0.034918
kim4,-0.100174,0.018409,0.058266,-0.080150,-0.108256,-0.051156,-0.087436,-0.120186,0.028922,-0.170093,...,0.142973,0.100357,0.003116,0.006549,-0.198405,-0.037609,0.179810,-0.000526,0.096610,-0.008519
kim5,-0.091822,0.031022,0.069909,-0.088182,-0.150732,-0.048896,-0.048058,-0.147652,0.059332,-0.126718,...,0.109271,0.074506,-0.042370,0.061233,-0.185493,-0.043759,0.135925,-0.002170,0.070462,-0.012326
kim6,-0.100594,0.032856,0.047395,-0.085442,-0.062414,-0.080291,-0.103689,-0.138430,0.112533,-0.117098,...,0.158315,0.114222,-0.018195,0.036954,-0.196113,-0.025679,0.116657,-0.009947,0.122645,-0.023930
kim7,-0.141279,0.048912,0.047157,-0.140672,-0.160152,-0.018800,-0.106201,-0.078768,0.114847,-0.188819,...,0.105087,0.090598,-0.045077,0.003897,-0.263763,-0.033162,0.154792,0.024102,0.088644,0.021292
kim8,-0.089850,0.096005,0.041754,-0.052878,-0.094131,0.027970,-0.116759,-0.078475,0.174731,-0.163058,...,0.136254,0.108000,-0.029237,-0.003817,-0.210180,0.007962,0.089843,-0.071499,0.115876,-0.006083
kim9,-0.074607,0.043346,0.093928,-0.107175,-0.102226,-0.059615,-0.074609,-0.148097,0.136101,-0.160114,...,0.116255,0.082958,-0.035539,0.003622,-0.160167,0.003947,0.112970,0.025309,0.095867,-0.010663


# -----------------------------

In [466]:
len(load_embedding_csv.index)

9

In [484]:
test1.face_database['kim10']

[array([-8.40628743e-02,  3.97305042e-02,  4.88503128e-02, -1.39993519e-01,
        -1.32822156e-01, -3.05293053e-02, -7.96421468e-02, -1.33434772e-01,
         1.07182980e-01, -1.69251412e-01,  1.98322132e-01, -1.12051889e-01,
        -2.53489286e-01, -3.63074951e-02, -3.42699140e-02,  2.43878439e-01,
        -1.81310177e-01, -1.49393141e-01, -7.96695203e-02,  1.79768205e-02,
         3.94068062e-02,  1.76175237e-02,  4.38323021e-02,  7.79714882e-02,
        -1.35108948e-01, -3.20007861e-01, -1.07441083e-01, -4.41897511e-02,
        -9.89553332e-02, -5.37203401e-02, -5.24426028e-02, -6.43511117e-03,
        -2.05156639e-01, -6.58197701e-03,  4.11223508e-02,  7.76233301e-02,
         1.76398959e-02, -1.43493816e-01,  1.45892799e-01,  2.30009593e-02,
        -3.07533354e-01,  7.03573301e-02,  8.23154822e-02,  1.94742143e-01,
         1.68915778e-01,  1.91420317e-04, -8.96685198e-03, -1.27614915e-01,
         1.46026522e-01, -1.64016321e-01,  9.54418257e-03,  9.82426852e-02,
         3.0

In [490]:
[load_embedding_csv.iloc[0].values]

[array([-8.40628743e-02,  3.97305042e-02,  4.88503128e-02, -1.39993519e-01,
        -1.32822156e-01, -3.05293053e-02, -7.96421468e-02, -1.33434772e-01,
         1.07182980e-01, -1.69251412e-01,  1.98322132e-01, -1.12051889e-01,
        -2.53489286e-01, -3.63074951e-02, -3.42699140e-02,  2.43878439e-01,
        -1.81310177e-01, -1.49393141e-01, -7.96695203e-02,  1.79768205e-02,
         3.94068062e-02,  1.76175237e-02,  4.38323021e-02,  7.79714882e-02,
        -1.35108948e-01, -3.20007861e-01, -1.07441083e-01, -4.41897511e-02,
        -9.89553332e-02, -5.37203401e-02, -5.24426028e-02, -6.43511117e-03,
        -2.05156639e-01, -6.58197701e-03,  4.11223508e-02,  7.76233301e-02,
         1.76398959e-02, -1.43493816e-01,  1.45892799e-01,  2.30009593e-02,
        -3.07533354e-01,  7.03573301e-02,  8.23154822e-02,  1.94742143e-01,
         1.68915778e-01,  1.91420317e-04, -8.96685198e-03, -1.27614915e-01,
         1.46026522e-01, -1.64016321e-01,  9.54418257e-03,  9.82426852e-02,
         3.0

In [496]:
bool_list = []
for i in range(len(load_embedding_csv.index)):
    _, compare_check = test1.compare_img(emb1,  [load_embedding_csv.iloc[i].values], False)
    print(compare_check)
    bool_list.extend(compare_check)
   

[0.44629841]
[True]
[1.02122406e-16]
[True]
[0.39631439]
[True]
[0.39924702]
[True]
[0.4130715]
[True]
[0.50079765]
[True]
[0.47429415]
[True]
[0.60493536]
[False]
[0.43149893]
[True]


In [497]:
bool_list

[True, True, True, True, True, True, True, False, True]

In [462]:
load_embedding_csv

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
kim10,-0.084063,0.039731,0.048850,-0.139994,-0.132822,-0.030529,-0.079642,-0.133435,0.107183,-0.169251,...,0.131356,0.068678,-0.021161,0.044097,-0.167169,-0.023806,0.135986,-0.001535,0.121353,0.006514
kim2,-0.099417,0.086221,0.078085,-0.064725,-0.112766,-0.038902,-0.035343,-0.149757,0.097881,-0.174078,...,0.122097,0.083155,-0.029671,0.049072,-0.197699,-0.003710,0.147324,0.024033,0.092897,0.002944
kim3,-0.101092,0.052281,0.072207,-0.138972,-0.104183,-0.027339,-0.082474,-0.149408,0.124288,-0.177972,...,0.133492,0.100071,-0.002764,0.015872,-0.208168,-0.013111,0.141747,-0.008849,0.080920,-0.034918
kim4,-0.100174,0.018409,0.058266,-0.080150,-0.108256,-0.051156,-0.087436,-0.120186,0.028922,-0.170093,...,0.142973,0.100357,0.003116,0.006549,-0.198405,-0.037609,0.179810,-0.000526,0.096610,-0.008519
kim5,-0.091822,0.031022,0.069909,-0.088182,-0.150732,-0.048896,-0.048058,-0.147652,0.059332,-0.126718,...,0.109271,0.074506,-0.042370,0.061233,-0.185493,-0.043759,0.135925,-0.002170,0.070462,-0.012326
kim6,-0.100594,0.032856,0.047395,-0.085442,-0.062414,-0.080291,-0.103689,-0.138430,0.112533,-0.117098,...,0.158315,0.114222,-0.018195,0.036954,-0.196113,-0.025679,0.116657,-0.009947,0.122645,-0.023930
kim7,-0.141279,0.048912,0.047157,-0.140672,-0.160152,-0.018800,-0.106201,-0.078768,0.114847,-0.188819,...,0.105087,0.090598,-0.045077,0.003897,-0.263763,-0.033162,0.154792,0.024102,0.088644,0.021292
kim8,-0.089850,0.096005,0.041754,-0.052878,-0.094131,0.027970,-0.116759,-0.078475,0.174731,-0.163058,...,0.136254,0.108000,-0.029237,-0.003817,-0.210180,0.007962,0.089843,-0.071499,0.115876,-0.006083
kim9,-0.074607,0.043346,0.093928,-0.107175,-0.102226,-0.059615,-0.074609,-0.148097,0.136101,-0.160114,...,0.116255,0.082958,-0.035539,0.003622,-0.160167,0.003947,0.112970,0.025309,0.095867,-0.010663
